In [19]:
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.keras import optimizers
from tensorflow.python.keras import activations
from tensorflow.python.keras import callbacks
from tensorflow.python.keras import initializers
from tensorflow.python.keras import backend
from tensorflow.python.keras import metrics
from tensorflow.python import keras
import tensorflow as tf
import keras_tuner as kt
import numpy as np
from sklearn.model_selection import train_test_split

In [24]:
train_x_rc = np.load('../data/train_x_rc.npy')
train_x_other = np.load('../data/train_x_other.npy')
targets = np.load('../data/target.npy')
print(train_x_other.shape)
print(train_x_rc.shape)
print(targets.shape)

(75450, 80, 10)
(75450, 80, 15)
(75450, 80)


In [25]:
rc_num = 15
ox = 10
train_x = np.concatenate([train_x_rc, train_x_other], axis=-1)
x_train, x_test, y_train, y_test = train_test_split(train_x, targets, test_size=0.5, random_state=43)
x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size=0.75, random_state=43)
x_train_rc = x_train[:, :, :rc_num][:, 0]
x_train_ox = x_train[:, :, -ox:]
x_test_rc = x_test[:, :, :rc_num][:, 0]
x_test_ox = x_test[:, :, -ox:]
x_valid_rc = x_valid[:, :, :rc_num][:, 0]
x_valid_ox = x_valid[:, :, -ox:]

In [26]:
class ExpandTileLayer(layers.Layer):
    def __init__(self):
        super(ExpandTileLayer, self).__init__()
    def call(self, inputs, *args, **kwargs):
        return backend.tile(backend.expand_dims(inputs, axis=-2), (1, 80, 1))

In [27]:
def build_model(hp):
    rc_input = keras.Input(shape=(train_x_rc.shape[-1], ),
                           dtype='int32',
                           name='rc_input_layer')
    other_x_input = keras.Input(shape=(80, train_x_other.shape[-1]),
                                dtype='float32',
                                name='other_x_input')
    rc_embedding_layer = layers.Dense(units=hp.Choice('units_emb', [8, 16, 32, 64]),
                                      use_bias=False, name='rc_embed_layer')
    rc_embedding = rc_embedding_layer(rc_input)

    # 维度拓展
    rc_embedding = ExpandTileLayer()(rc_embedding)

    x_input = layers.Concatenate(axis=-1)([other_x_input, rc_embedding])
    ox = layers.Bidirectional(layers.LSTM(units=hp.Choice('units_0', [64, 128, 256]),
                                          return_sequences=True,
                                          kernel_initializer=initializers.initializers_v2.GlorotUniform()),
                              merge_mode='concat')(x_input)
    for i in range(hp.Int('num_layers', min_value=1, max_value=3, step=1)):
        ox = layers.Bidirectional(layers.LSTM(units=hp.Choice('units_'+str(i), [64, 128, 256]),
                                              return_sequences=True,
                                              kernel_initializer=initializers.initializers_v2.GlorotUniform()),
                                  merge_mode='concat')(ox)
    ox = layers.Dense(units=hp.Choice('units_d1', [64, 128, 256]),
                      kernel_initializer=initializers.initializers_v2.GlorotUniform())(ox)
    if hp.Choice('d2_activation', ['prelu', 'elu']) == 'prelu':
        lstm_output = layers.PReLU()(ox)
    else:
        lstm_output = layers.ELU()(ox)

    conv1d_1 = layers.Conv1D(filters=hp.Choice('conv1d_1_filter', [64, 128, 256]),
                             kernel_size=hp.Choice('cov1d_1_size', [5, 11]),
                             padding='same', use_bias=False)(x_input)
    conv1d_1 = layers.BatchNormalization()(conv1d_1)
    conv1d_1 = layers.ReLU()(conv1d_1)
    conv1d_2 = layers.Conv1D(filters=64, padding='same', use_bias=False,
                             kernel_size=hp.Choice('conv1d_2_size', [5, 7]))(conv1d_1)
    conv1d_2 = layers.BatchNormalization()(conv1d_2)
    conv1d_output = layers.ReLU()(conv1d_2)

    output = layers.Concatenate(axis=-1)([lstm_output, conv1d_output])
    output = layers.Dense(units=hp.Choice('final_units', [128, 256]),
                          activation=activations.relu,
                          kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)
    output = layers.Dense(units=1,
                          kernel_initializer=initializers.initializers_v2.GlorotUniform())(output)

    my_model = models.Model(inputs=[rc_input, other_x_input], outputs=[output])
    my_model.compile(loss=losses.MeanAbsoluteError(),
                     optimizer=optimizers.Adam(),
                     metrics=[metrics.MeanAbsoluteError()])
    return my_model

In [28]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=2,
    overwrite=True,
    directory='../tuner',
    project_name='ventilator'
)

In [29]:
print(tuner.search_space_summary())

Search space summary
Default search space size: 9
units_emb (Choice)
{'default': 8, 'conditions': [], 'values': [8, 16, 32, 64], 'ordered': True}
units_0 (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 256], 'ordered': True}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units_d1 (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 256], 'ordered': True}
d2_activation (Choice)
{'default': 'prelu', 'conditions': [], 'values': ['prelu', 'elu'], 'ordered': False}
conv1d_1_filter (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 256], 'ordered': True}
cov1d_1_size (Choice)
{'default': 5, 'conditions': [], 'values': [5, 11], 'ordered': True}
conv1d_2_size (Choice)
{'default': 5, 'conditions': [], 'values': [5, 7], 'ordered': True}
final_units (Choice)
{'default': 128, 'conditions': [], 'values': [128, 256], 'ordered': True}
None


In [32]:
tuner.search([x_train_rc, x_train_ox], y_train,
             validation_data=([x_test_rc, x_test_ox], y_test),
             epochs=25,
             batch_size=64)


Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
units_emb         |32                |?                 
units_0           |256               |?                 
num_layers        |2                 |?                 
units_d1          |128               |?                 
d2_activation     |prelu             |?                 
conv1d_1_filter   |256               |?                 
cov1d_1_size      |11                |?                 
conv1d_2_size     |7                 |?                 
final_units       |128               |?                 
units_1           |64                |?                 

Epoch 1/25
  3/590 [..............................] - ETA: 48:03 - loss: 10.4554 - mean_absolute_error: 10.4554  

KeyboardInterrupt: 